## Projektarbeit Mühle

### Imports

In [33]:

from collections import defaultdict
import PySimpleGUI as psGui
import numpy as np
import time
import copy
from tensorflow import keras
import tensorflow as tf
import datetime as dt
import sklearn.preprocessing as pre
from typing import List
import math

### Class definitions

#### Environment

In [59]:

class MillEnv(object):
    def __init__(self):
        self.isPlaying: int = 1
        self.gamePhase: list = [0,0]
        self.moveNeeded: int = 0
        self.inHand: list = [9,9]
        self.onBoard: list = [0,0]
        self.checkerPositions: list = [[],[]]
        self.selected: int = -1
        self.board: np.ndarray = np.zeros(24)
        self.winner = 0
        self.columns: np.ndarray = np.array([[0,1,2], # for determining how many checkers from each player are in a row
                        [3,4,5],
                        [6,7,8],
                        [9,10,11],
                        [12,13,14],
                        [15,16,17],
                        [18,19,20],
                        [21,22,23],
                        [0,9,21],
                        [3,10,18],
                        [6,11,15],
                        [1,4,7],
                        [16,19,22],
                        [8,12,17],
                        [5,13,20],
                        [2,14,23],
                        ])
    def makeMove(self, move: int) -> bool:
        valid: bool = False
        last_state: tuple = self.getSummary(self.isPlaying)
        last_player: int = self.isPlaying
        if self.moveNeeded == 0: # Set Checker on position
            if self.board[move] == 0:
                self.board[move] = self.isPlaying
                self.checkerPositions[1 if self.isPlaying == 1 else 0].append(move)
                valid = True
                if self.gamePhase[1 if self.isPlaying == 1 else 0] == 2:
                    self.board[self.selected] = 0
                    self.moveNeeded = 1
                    self.checkerPositions[1 if self.isPlaying == 1 else 0].remove(self.selected)
                else:
                    self.inHand[1 if self.isPlaying == 1 else 0] -= 1
                    self.onBoard[1 if self.isPlaying == 1 else 0] += 1
                    if np.array(self.inHand).sum() == 0:
                        self.gamePhase = [1,1]
                        self.moveNeeded = 1
                self.isPlaying = -self.isPlaying
        elif self.moveNeeded == 1: # choose checker to move
            if self.board[move] == 1 * self.isPlaying and (~(self.getMoveFields(move).all()) or self.gamePhase[1 if self.isPlaying == 1 else 0] == 2):
                valid = True
                self.selected = move
                if self.gamePhase[1 if self.isPlaying == 1 else 0] == 2:
                    self.moveNeeded = 0
                else:
                    self.moveNeeded = 2
        elif self.moveNeeded == 2: # move checker up, down, left or right
            if self.getMoveFields(self.selected)[move] == 0:
                idxToMoveAxis: np.ndarray = np.where(self.getInRows(self.selected) == self.selected)
                idxToMove = list(zip(idxToMoveAxis[1], idxToMoveAxis[0]))
                order = idxToMove[0][1]
                valid = True
                self.board[self.selected] = 0
                last_state = self.getSummary(last_player)
                self.checkerPositions[1 if self.isPlaying == 1 else 0].remove(self.selected)
                if move == 0: # up
                    self.board[self.getInRows(self.selected)[1][idxToMove[abs(order-1)][0]-1]] = self.isPlaying
                    self.checkerPositions[1 if self.isPlaying == 1 else 0].append(self.getInRows(self.selected)[1][idxToMove[abs(order-1)][0]-1])
                if move == 1: # right
                    self.board[self.getInRows(self.selected)[0][idxToMove[order][0]+1]] = self.isPlaying
                    self.checkerPositions[1 if self.isPlaying == 1 else 0].append(self.getInRows(self.selected)[0][idxToMove[order][0]+1])
                if move == 2: # down
                    self.board[self.getInRows(self.selected)[1][idxToMove[abs(order-1)][0]+1]] = self.isPlaying
                    self.checkerPositions[1 if self.isPlaying == 1 else 0].append(self.getInRows(self.selected)[1][idxToMove[abs(order-1)][0]+1])
                if move == 3: # left
                    self.board[self.getInRows(self.selected)[0][idxToMove[order][0]-1]] = self.isPlaying
                    self.checkerPositions[1 if self.isPlaying == 1 else 0].append(self.getInRows(self.selected)[0][idxToMove[order][0]-1])
                self.selected = -1
                self.moveNeeded = 1
                self.isPlaying = -self.isPlaying
        elif self.moveNeeded == 3: # delete opponent checker
            threeInChosenRow: np.ndarray = abs(self.board[self.getInRows(move)].sum(axis=1)) == 3
            if self.board[move] == -1 * self.isPlaying and ~threeInChosenRow.any():
                valid = True
                self.board[move] = 0
                self.checkerPositions[1 if self.isPlaying == -1 else 0].remove(move)
                self.onBoard[1 if self.isPlaying == -1 else 0] -= 1
                if self.gamePhase[1 if self.isPlaying == -1 else 0] == 0:
                    self.moveNeeded = 0
                elif self.gamePhase[1 if self.isPlaying == -1 else 0] == 1:
                    if self.onBoard[1 if self.isPlaying == -1 else 0] == 3:
                        self.gamePhase[1 if self.isPlaying == -1 else 0] = 2
                    self.moveNeeded = 1
                elif self.gamePhase[1 if self.isPlaying == -1 else 0] == 2:
                    self.gamePhase = [3,3]
                    self.winner = last_player
                self.isPlaying = -self.isPlaying
        if last_state[0] < self.getSummary(last_player)[0]:
            self.isPlaying = last_player
            self.moveNeeded = 3
            canDelete: bool = False
            for pos in self.checkerPositions[1 if self.isPlaying == -1 else 0]:
                if ~(abs(self.board[self.getInRows(pos)].sum(axis=1)) == 3).any():
                    canDelete = True
                    break
            if not canDelete:
                valid = True
                if self.gamePhase[1 if self.isPlaying == -1 else 0] == 0:
                    self.moveNeeded = 0
                elif self.gamePhase[1 if self.isPlaying == -1 else 0] >= 1:
                    self.moveNeeded = 1
        if self.gamePhase[1 if last_player == -1 else 0] == 1:
            finished = True
            for pos in self.checkerPositions[1 if last_player == -1 else 0]:
                if ~(self.getMoveFields(pos).all()):
                    finished = False
                    break
            if finished:
                self.winner = last_player
                self.gamePhase = [3,3]
        return valid
    def isFinished(self):
        return self.winner
    def getBoard(self) -> np.ndarray:
        return copy.deepcopy(self.board)
    def getInRows(self, pos: int) -> np.ndarray:
        arrayPos: np.ndarray = self.columns == pos
        return self.columns[arrayPos.any(axis=1)]
    def reset(self):
        self.isPlaying: int = 1
        self.gamePhase: list = [0,0]
        self.moveNeeded: int = 0
        self.inHand: list = [9,9]
        self.onBoard: list = [0,0]
        self.checkerPositions: list = [[],[]]
        self.selected: int = -1
        self.board: np.ndarray = np.zeros(24)
        self.winner = 0
    def getSummary(self, player: int) -> (int, int, int, int):
        numTwoPlayerActual: int = 0
        numTwoPlayerOpponent: int = 0
        numThreePlayerActual: int = 0
        numThreePlayerOpponent: int = 0
        for column in self.columns:
            columnSum: int = self.board[column[0]] + self.board[column[1]] + self.board[column[2]]
            if columnSum == -2 * player:
                numTwoPlayerOpponent += 1
            elif columnSum  == 2 * player:
                numTwoPlayerActual += 1
            elif columnSum == -3 * player:
                numTwoPlayerOpponent += 1
            elif columnSum == 3 * player:
                numThreePlayerActual += 1
        return numThreePlayerActual, numThreePlayerOpponent, numTwoPlayerActual, numTwoPlayerOpponent
    def getMoveFields(self, pos: int) -> np.ndarray:
        moveFields: np.ndarray = np.zeros(4)
        chosenRows: np.ndarray = self.getInRows(pos)
        idxAxis: np.ndarray = np.where(chosenRows == pos)
        idx = list(zip(idxAxis[1], idxAxis[0]))
        order = idx[0][1]
        if idx[order][0] != 1:
            if idx[order][0] == 0:
                moveFields[3] = 2
                moveFields[1] = self.board[chosenRows[0][1]]
            elif idx[order][0] == 2:
                moveFields[1] = 2
                moveFields[3] = self.board[chosenRows[0][1]]
        else:
            moveFields[3] = self.board[chosenRows[0][0]]
            moveFields[1] = self.board[chosenRows[0][2]]
        if idx[abs(order-1)][0] != 1:
            if idx[abs(order-1)][0] == 0:
                moveFields[0] = 2
                moveFields[2] = self.board[chosenRows[1][1]]
            elif idx[abs(order-1)][0] == 2:
                moveFields[2] = 2
                moveFields[0] = self.board[chosenRows[1][1]]
        else:
            moveFields[0] = self.board[chosenRows[1][0]]
            moveFields[2] = self.board[chosenRows[1][2]]
        return moveFields
    def getValidMoves(self) -> List[int]:
        validList = []
        if self.moveNeeded == 0:
            validList = np.arange(24)
            validList = np.setdiff1d(validList,self.checkerPositions[0])
            validList = np.setdiff1d(validList,self.checkerPositions[1])
        elif self.moveNeeded == 1:
            validList = [pos for pos in self.checkerPositions[1 if self.isPlaying == 1 else 0] if ~self.getMoveFields(pos).all() or self.gamePhase[1 if self.isPlaying == 1 else 0] == 2]
        elif self.moveNeeded == 2:
            validList = [move for move in np.arange(4) if self.getMoveFields(self.selected)[move] == 0]
        elif self.moveNeeded == 3:
            validList = [pos for pos in self.checkerPositions[1 if self.isPlaying == -1 else 0] if ~(abs(self.board[self.getInRows(pos)].sum(axis=1)) == 3).any()]
        return validList
    def getReward(self, last_state, new_state, valid, to_train) -> float:
        reward = 0.
        if not valid:
                return -8.
        if to_train == 0 or to_train == 2:
            if last_state[0] < new_state[0]:
                reward += 20.
            if last_state[2] < new_state[2]:
                reward += 5. * (new_state[2] - last_state[2])
            if last_state[3] > new_state[3]:
                reward += 10. * (last_state[3] - new_state[3])
        elif to_train == 1:
            if valid:
                return 5.
        elif to_train == 3:
            if last_state[2] < new_state[2]:
                return 15.
            elif last_state[3] > new_state[3]:
                return 15.
        return reward
    def getFullState(self):
        return copy.deepcopy((self.getBoard(), self.isPlaying, self.gamePhase, self.moveNeeded,self.inHand, self.onBoard, self.checkerPositions, self.selected, self.winner))
    def setFullState(self, board, isPlaying, gamePhase, moveNeded, inHand, onboard, checkerPositions, selected, winner):
        self.isPlaying: int = copy.deepcopy(isPlaying)
        self.gamePhase: list = copy.deepcopy(gamePhase)
        self.moveNeeded: int = copy.deepcopy(moveNeded)
        self.inHand: list = copy.deepcopy(inHand)
        self.onBoard: list = copy.deepcopy(onboard)
        self.checkerPositions: list = copy.deepcopy(checkerPositions)
        self.selected: int = copy.deepcopy(selected)
        self.board: np.ndarray = copy.deepcopy(board)
        self.winner = copy.deepcopy(winner)

#### Agent

In [35]:
class Agent(object):
    def __init__(self):
        self.encoder = pre.OneHotEncoder(sparse=False).fit(np.array([1,0,-1]).reshape(-1,1))
    def processState(self, player, state, moveNeeded, selected= None):
        state = state * player
        state_OH = np.array(self.encoder.transform(state.reshape(-1,1)))
        if moveNeeded == 2 and selected is not None:
            state[selected] = 2
            selectedArray = np.zeros(24)
            selectedArray[selected] = 1
            # TODO append selectedArray to state_OH
            state_OH = np.append(state_OH, selectedArray.reshape(-1,1), axis=1)
        return state_OH.flatten()
        #return state
    def getPos(self, state: np.ndarray, temp: float, validMoves: List[int],network:keras.Model=None) -> np.ndarray:
        if network is None:
            return np.random.choice(validMoves)
        softmaxed_output = keras.backend.softmax(network(state.reshape(1,-1))/ temp)
        action_value = np.random.choice(np.array(softmaxed_output[0]), p= np.array(softmaxed_output[0]))
        pos: np.ndarray = np.argmax(softmaxed_output[0] == action_value)
        return pos


#### Graphics

In [36]:
class MillDisplayer(object):
    def __init__(self, MillEnvironment: MillEnv = None):
        psGui.theme("dark")
        self.millImage: str = "MühleBrett.png"
        self.blackCheckerImage: str  = "Schwarz.png"
        self.whiteCheckerImage: str = "Weiss.png"
        self.millEnv: MillEnv = MillEnv()
        if MillEnvironment is not None :
            self.millEnv = MillEnvironment
        self.ImageIDArray = np.array([])
        self.imageLocations = [(10,490), (225, 490), (440, 490),
                                (75,415), (225, 415), (375, 415),
                                (150,340), (225, 340), (310, 340),
                                (10,265), (75, 265), (150, 265),
                                (310,265), (375, 265), (440, 265),
                                (150,190), (225, 190), (310, 190),
                                (75,115), (225, 115), (375, 115),
                                (10,55), (225, 55), (440, 55)]
        self.graph = psGui.Graph(
                        canvas_size=(500, 500),
                        graph_bottom_left=(0, 0),
                        graph_top_right=(500, 500),
                        )
        self.statusTextBox = psGui.Text("Player "+self.getPlayerName(self.millEnv.isPlaying)+" is playing", size=(50, 1))
        self.layout_ = [[psGui.Button("Player vs. Player"),psGui.Button("Player vs. Agent"),psGui.Button("Agent vs. Agent")],
                        [self.statusTextBox],
                       [self.graph],
                       [psGui.Button("Close")]]
        self.window  = psGui.Window("Mill AI", layout=self.layout_)
        self.window.finalize()
        self.graph.DrawImage(filename=self.millImage, location=(0,500))
        self.activateClick()
        self.reloadEnv()
    def windowsLoop(self):
        while True:
            event, values = self.window.read()
            if event == psGui.WIN_CLOSED or event == 'Close': # if user closes window or clicks cancel
                break
            elif not event == "":
                self.reset()
        self.window.close()
    def makeMove(self, pos: int) -> bool:
        valid: bool = self.millEnv.makeMove(pos)
        if valid:
            self.reloadEnv()
        return valid
    def reloadEnv(self):
        self.setStatus("Player " + self.getPlayerName(self.millEnv.isPlaying) + " is playing - move needed: " + str(self.millEnv.moveNeeded))
        for imageID in self.ImageIDArray:
            self.graph.DeleteFigure(imageID)
        np.delete(self.ImageIDArray, np.s_[:])
        for case, location in zip(self.millEnv.getBoard(), self.imageLocations):
            if case == 1:
                self.ImageIDArray = np.append(self.ImageIDArray,self.graph.DrawImage(filename=self.blackCheckerImage, location=location))
            elif case == -1:
                self.ImageIDArray = np.append(self.ImageIDArray,self.graph.DrawImage(filename=self.whiteCheckerImage, location=location))
        self.window.refresh()
    def getClicked(self, event) -> int:
        for index, location in enumerate(self.imageLocations):
            x2, y2 = location
            if self.isInArea(event.x, -event.y + 500, x2, y2, 50, 50):
                return index
        return -1
    def setAfterClicked(self, event):
        pos = self.getClicked(event)
        if pos == -1:
            return False
        if self.millEnv.moveNeeded == 2:
            dif = self.millEnv.selected - pos
            if dif == 0:
                return False
            if dif == -1:
                pos = 1
            elif dif == 1:
                pos = 3
            elif dif < 0:
                pos = 2
            elif dif > 0:
                pos = 0
        return self.makeMove(pos)
    def isInArea(self, posX1: int, posY1: int, posX2: int, posY2: int, width: int, height: int) -> bool:
        if posX2 <= posX1 <= posX2 + width:
            if posY2 >= posY1 >= posY2 - height:
                return True
        return False
    def setStatus(self, status: str):
        self.statusTextBox.Update(status)
    def close(self):
        self.window.close()
    def activateClick(self):
        self.graph.TKCanvas.bind("<Button-1>",self.setAfterClicked)
    def deactivateClick(self):
        self.graph.TKCanvas.unbind("<Button-1>")
    def read(self, timout: bool=False):
        return self.window.read(1 if timout else None)
    def reset(self):
        self.millEnv.reset()
        self.reloadEnv()
    def getPlayerName(self, player: int) -> str:
        if player == 1:
            return "black"
        elif player == -1:
            return "white"
        else:
            return "not a player"


#### Moderated Graphics

In [53]:
class ModeratedGraphics(object):
    def __init__(self, modelList: List[keras.Model] = [None,None,None,None], monteCarlo = False, gamma=0.8):
        self.gamma = gamma
        self.env = MillEnv()
        self.graphics = MillDisplayer(self.env)
        self.graphics.reloadEnv()
        self.monteCarlo = monteCarlo
        if not self.monteCarlo:
            self.modelList = modelList
            self.agent = Agent()
        else:
            self.root: State = State(self.env.getFullState(), self.env.getValidMoves(), self.env.isFinished(), self.env)
            self.mcts = MonteCarloTreeSearch(self.root)
    def agentPlay(self):
        if self.monteCarlo:
            self.resetMonteCarlo()
        else:
            self.graphics.reset()
        self.graphics.deactivateClick()
        finished = 0
        while finished == 0:
            if not self.monteCarlo:
                pos = self.agent.getPos(self.env.getBoard(),1, self.env.getValidMoves(),self.modelList[self.env.moveNeeded])
            else:
                self.root = self.mcts.best_action(self.gamma,simulations_number=5000, max_depth=10)
                pos = self.root.last_move
                self.mcts.setNewRoot(self.root)
            self.graphics.makeMove(pos)
            event, values = self.graphics.read(True)
            if self.eventHandler(event):
                return
            finished = self.env.isFinished()
        if not finished == 2:
            self.graphics.setStatus("player " + self.graphics.getPlayerName(finished) +" won")
        else:
            self.graphics.setStatus("The game ended in a draw")
    def playersVSPlayer(self):
        self.graphics.activateClick()
        self.graphics.reset()
        finished = 0
        while finished == 0:
            event, values = self.graphics.read(True)
            if self.eventHandler(event):
                return
            self.graphics.reloadEnv()
            finished = self.env.isFinished()
        if not finished == 2:
            self.graphics.setStatus("player " + self.graphics.getPlayerName(finished) +" won")
        else:
            self.graphics.setStatus("The game ended in a draw")
        self.graphics.deactivateClick()
    def playerVSAgent(self):
        self.graphics.activateClick()
        if self.monteCarlo:
            self.resetMonteCarlo()
        else:
            self.graphics.reset()
        finished = 0
        while finished == 0:
            event, values = self.graphics.read(True)
            if self.eventHandler(event):
                return
            elif self.env.isPlaying == 1:
                self.graphics.activateClick()
            else:
                self.graphics.deactivateClick()
                if not self.monteCarlo:
                    pos = self.agent.getPos(self.env.getBoard(),1, self.env.getValidMoves(),self.modelList[self.env.moveNeeded])
                else:
                    self.root = State(self.env.getFullState(), self.env.getValidMoves(), self.env.isFinished(), self.env)
                    self.mcts.setNewRoot(self.root)
                    self.root = self.mcts.best_action(self.gamma,simulations_number=5000, max_depth=10)
                    pos = self.root.last_move
                    self.mcts.setNewRoot(self.root)
                self.graphics.makeMove(pos)
            self.graphics.reloadEnv()
            finished = self.env.isFinished()
        if not finished == 2:
            self.graphics.setStatus("player " + self.graphics.getPlayerName(finished) +" won")
        else:
            self.graphics.setStatus("The game ended in a draw")
        self.graphics.deactivateClick()
    def playLoop(self):
        self.graphics.deactivateClick()
        self.playerVSAgent()
        finished = False
        while not finished:
            event, values = self.graphics.read()
            finished = self.eventHandler(event)
    def eventHandler(self, event) -> bool:
        if event == psGui.WIN_CLOSED or event == 'Close': # if user closes window or clicks cancel
            self.graphics.close()
            return True
        elif event == "Agent vs. Agent":
            self.agentPlay()
        elif event == "Player vs. Player":
            self.playersVSPlayer()
        elif event == "Player vs. Agent":
            self.playerVSAgent()
        return False
    def resetMonteCarlo(self):
        self.env.reset()
        self.root = State(self.env.getFullState(), self.env.getValidMoves(), self.env.isFinished(), copy.deepcopy(self.env))
        self.mcts.setNewRoot(self.root)

#### Memory

In [38]:
class Memory(object):
    def __init__(self, size: int):
        self.size = size
        self.curr_write_idx = 0
        self.available_samples = 0
        self.buffer = np.array([(np.zeros(24,dtype=np.float32), 0.0, 0.0, np.zeros(24,
                                dtype=np.float32), False) for index in range(self.size)], dtype=object)
        self.base_node, self.leaf_nodes = create_tree([0 for index in range(self.size)])
        self.frame_idx = 0
        self.action_idx = 1
        self.reward_idx = 2
        self.terminal_idx = 3
        self.beta = 0.4
        self.alpha = 0.6
        self.min_priority = 0.01

    def append(self, experience: tuple, priority: float):
        self.buffer[self.curr_write_idx] = experience
        self.update(self.curr_write_idx, priority)
        self.curr_write_idx += 1
        # reset the current writer position index if creater than the allowed size
        if self.curr_write_idx >= self.size:
            self.curr_write_idx = 0
        # max out available samples at the memory buffer size
        if self.available_samples + 1 < self.size:
            self.available_samples += 1
        else:
            self.available_samples = self.size - 1

    def update(self, idx: int, priority: float):
        update(self.leaf_nodes[idx], self.adjust_priority(priority))

    def adjust_priority(self, priority: float):
        return np.power(priority + self.min_priority, self.alpha)

    def sample(self, num_samples: int):
        sampled_idxs = []
        is_weights = []
        sample_no = 0
        while sample_no < num_samples:
            sample_val = np.random.uniform(0, self.base_node.value)
            samp_node = retrieve(sample_val, self.base_node)
            if samp_node.idx < self.available_samples - 1:
                sampled_idxs.append(samp_node.idx)
                p = samp_node.value / self.base_node.value
                is_weights.append((self.available_samples + 1) * p)
                sample_no += 1
        # apply the beta factor and normalise so that the maximum is_weight < 1
        is_weights = np.array(is_weights)
        is_weights = np.power(is_weights, -self.beta)
        is_weights = is_weights / np.max(is_weights)
        # now load up the state and next state variables according to sampled idxs
        return self.buffer[sampled_idxs], sampled_idxs, is_weights


#### Model

In [39]:
class DQModel(keras.Model):
    def __init__(self, hidden_size: int, num_actions: int, dueling: bool, dropoutRate: float):
        super(DQModel, self).__init__()
        self.dueling = dueling
        self.dropoutRate = dropoutRate
        self.dense1 = keras.layers.Dense(hidden_size *3, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.dense2 = keras.layers.Dense(hidden_size * 4, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.dense3 = keras.layers.Dense(hidden_size * 5, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.dense4 = keras.layers.Dense(hidden_size * 4, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.adv_dense1 = keras.layers.Dense(hidden_size * 3, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.adv_dense2 = keras.layers.Dense(hidden_size * 4, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.adv_dense3 = keras.layers.Dense(hidden_size * 3, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.adv_out = keras.layers.Dense(num_actions,
                                          kernel_initializer=keras.initializers.he_normal())
        if dueling:
            self.v_dense1 = keras.layers.Dense(hidden_size * 3, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
            self.v_dense2 = keras.layers.Dense(hidden_size * 4, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
            self.v_out = keras.layers.Dense(1, kernel_initializer=keras.initializers.he_normal())
            self.lambda_layer = keras.layers.Lambda(lambda x: x- tf.reduce_mean(x))
            self.combine = keras.layers.Add()

    def call(self, input, **kwargs):
        x = self.dense1(input)
        x = keras.backend.dropout(x, self.dropoutRate)
        x = self.dense2(x)
        x = keras.backend.dropout(x, self.dropoutRate)
        x = self.dense3(x)
        x = self.dense4(x)
        adv = self.adv_dense1(x)
        adv = self.adv_dense2(adv)
        adv = self.adv_dense3(adv)
        adv = self.adv_out(adv)
        if self.dueling:
            v = self.v_dense1(x)
            v = self.v_dense2(v)
            v = self.v_out(v)
            norm_adv = self.lambda_layer(adv)
            combined = self.combine([v, norm_adv])
            return combined
        return adv

    @tf.function
    def traceable(self, input, **kwargs):
        return self(input, **kwargs)

    def changeDropoutRate(self, rate):
        self.dropoutRate = rate

#### TreeNode

In [40]:
class Node:
    def __init__(self, left, right, is_leaf: bool = False, idx = None):
        self.left = left
        self.right = right
        self.is_leaf = is_leaf
        self.value = sum(n.value for n in (left, right) if n is not None)
        self.parent = None
        self.idx = idx  # this value is only set for leaf nodes
        if left is not None:
            left.parent = self
        if right is not None:
            right.parent = self

    @classmethod
    def create_leaf(cls, value, idx):
        leaf = cls(None, None, is_leaf=True, idx=idx)
        leaf.value = value
        return leaf


def create_tree(input: list):
    nodes = [Node.create_leaf(v, i) for i, v in enumerate(input)]
    leaf_nodes = nodes
    while len(nodes) > 1:
        inodes = iter(nodes)
        nodes = [Node(*pair) for pair in zip(inodes, inodes)]

    return nodes[0], leaf_nodes

def retrieve(value: float, node: Node):
    if node.is_leaf:
        return node

    if node.left.value >= value:
        return retrieve(value, node.left)
    else:
        return retrieve(value - node.left.value, node.right)

def update(node: Node, new_value: float):
    change = new_value - node.value

    node.value = new_value
    propagate_changes(change, node.parent)


def propagate_changes(change: float, node: Node):
    node.value += change

    if node.parent is not None:
        propagate_changes(change, node.parent)

#### Controler

In [41]:
def huber_loss(loss):
    return 0.5 * loss ** 2 if abs(loss) < 1.0 else abs(loss) - 0.5

class Controler(object):
    def __init__(self, min_dropout, max_dropout, dropout_min_iter, min_temp, max_temp, temp_min_iter, min_beta, max_beta, beta_min_iter,  gamma, batch_size, tau, num_actions, memory_samples, store_path, hidden_layers, delay_training, to_train):
        self.to_train = to_train
        self.delay_training = delay_training
        self.hidden_layers = hidden_layers
        self.store_path = store_path
        self.memory_samples = memory_samples
        self.tau = tau
        self.batch_size = batch_size
        self.gamma = gamma
        self.beta_min_iter = beta_min_iter
        self.max_beta = max_beta
        self.min_beta = min_beta
        self.temp_min_iter = temp_min_iter
        self.max_temp = max_temp
        self.min_temp = min_temp
        self.dropout_min_iter = dropout_min_iter
        self.max_dropout = max_dropout
        self.min_dropout = min_dropout
        self.num_actions = num_actions
        self.primary_network = DQModel(self.hidden_layers, self.num_actions, True, self.max_dropout)
        self.target_network = DQModel(self.hidden_layers, self.num_actions, True, 0.)
        self.primary_network.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.Huber())
        for t, e in zip(self.target_network.trainable_variables, self.primary_network.trainable_variables):
            t.assign(e)
        self.env = MillEnv()
        self.agent = Agent()
        self.memory = Memory(memory_samples)
        self.steps = 0
        self.temp = max_temp
        self.dropoutRate = max_dropout
        self.train_writer = tf.summary.create_file_writer("Tensorboard/" + self.store_path)
    def startGame(self):
        self.env.reset()
    def update_network(self):
        # update target network parameters slowly from primary network
        for t, e in zip(self.target_network.trainable_variables, self.primary_network.trainable_variables):
            t.assign(t * (1 - self.tau) + e * self.tau)
    def get_per_error(self, batch, primary_network, target_network):
        states = np.array([val[0] for val in batch])
        actions = np.array([val[1] for val in batch])
        rewards = np.array([val[2] for val in batch])
        next_states = np.array([val[3] for val in batch])
        valid_idxs = np.array([val[4] for val in batch])
        # predict Q(s,a) given the batch of states
        prim_qt = primary_network(states)
        # predict Q(s',a') from the evaluation network
        prim_qtp1 = primary_network(next_states)
        # copy the prim_qt tensor into the target_q tensor - we then will update one index corresponding to the max action
        target_q = prim_qt.numpy()
        updates = rewards
        batch_idxs = np.arange(len(batch))
        prim_action_tp1 = np.argmax(prim_qtp1.numpy(), axis=1)
        q_from_target = target_network(next_states)
        updates[valid_idxs] += self.gamma * q_from_target.numpy()[batch_idxs[valid_idxs], prim_action_tp1[valid_idxs]]
        target_q[batch_idxs, actions] = updates
        error = [huber_loss(target_q[i, actions[i]] - prim_qt.numpy()[i, actions[i]])for i in range(states.shape[0])]
        return target_q, error
    def train_network(self, primary_network, memory, target_network):
        batch, idxs, is_weights = memory.sample(self.batch_size)
        states = np.array([val[0] for val in batch])
        target_q, error = self.get_per_error(batch, primary_network, target_network )
        for i in range(len(idxs)):
            memory.update(idxs[i], error[i])
        loss = primary_network.train_on_batch(states, target_q, is_weights)
        return loss
    def gameLoop(self, episode):
        self.startGame()
        totreward = 0.
        last_state = [0,0]
        last_action = [0,0]
        reward = [0,0]
        cnt = 1
        avg_loss = 0
        invalid_moves = 0
        while True:
            if self.env.moveNeeded == self.to_train:
                self.primary_network.changeDropoutRate(self.dropoutRate)
                actual_player = self.env.isPlaying
                last_state[1 if actual_player == 1 else 0] = self.env.getBoard(), self.env.getSummary(actual_player)
                pos= self.agent.getPos(self.agent.processState(actual_player, self.env.getBoard(), self.to_train, self.env.selected), self.temp, self.env.getValidMoves(),self.primary_network)
                valid = self.env.makeMove(pos)
                reward[1 if actual_player == 1 else 0] = self.env.getReward(last_state[1 if actual_player == 1 else 0][1], self.env.getSummary(actual_player), valid, self.to_train)
                last_action[1 if actual_player == 1 else 0] = pos, valid
                finished = self.env.isFinished()
                if last_state[1 if actual_player == -1 else 0] != 0 and valid:
                    new_state = self.env.getBoard(), self.env.getSummary(-actual_player)
                    rewardAbs = self.env.getReward(last_state[1 if actual_player == -1 else 0][1], new_state[1], last_action[1 if actual_player == -1 else 0][1], self.to_train) if finished != 0 else reward[1 if actual_player == -1 else 0]
                    totreward += rewardAbs
                    sample = (self.agent.processState(-actual_player,last_state[1 if actual_player == -1 else 0][0], self.to_train, self.env.selected), last_action[1 if actual_player == -1 else 0][0], rewardAbs, self.agent.processState(-actual_player,new_state[0], self.to_train, self.env.selected), finished == 0)
                    _, error = self.get_per_error([sample], self.primary_network, self.target_network)
                    self.memory.append(sample,error[0])
                if self.steps > self.delay_training:
                    loss = self.train_network(self.primary_network, self.memory, self.target_network)
                    self.update_network()
                else:
                    loss = 0
                avg_loss += loss
                if self.steps > self.delay_training:
                    self.temp = self.max_temp - ((self.steps - self.delay_training) / self.temp_min_iter) * \
                      (self.max_temp - self.min_temp) if self.steps < self.temp_min_iter else \
                    self.min_temp
                    self.dropoutRate = self.max_dropout - ((self.steps - self.delay_training) / self.dropout_min_iter) * \
                      (self.max_dropout - self.min_dropout) if self.steps < self.dropout_min_iter else \
                    self.min_dropout
                    beta = self.min_beta + ((self.steps - self.delay_training) / self.beta_min_iter) * \
                      (self.max_beta - self.min_beta) if self.steps < self.beta_min_iter else \
                    self.max_beta
                    self.memory.beta = beta
                self.steps += 1
                if cnt >= 1000:
                    finished = 2
                if  not valid or finished != 0:
                    invalid_moves += 0 if finished != 0 else 1
                    new_state = self.env.getBoard()
                    totreward += reward[1 if actual_player == 1 else 0]
                    sample = (self.agent.processState(actual_player, last_state[1 if actual_player == 1 else 0][0], self.to_train, self.env.selected), last_action[1 if actual_player == 1 else 0][0], reward[1 if actual_player == 1 else 0],self.agent.processState(actual_player,new_state, self.to_train, self.env.selected), False)
                    _, error = self.get_per_error([sample], self.primary_network, self.target_network)
                    self.memory.append(sample, error[0])
                cnt += 1
            else:
                self.env.makeMove(self.agent.getPos(self.env.getBoard(), 1,self.env.getValidMoves()))
            if self.env.isFinished() != 0:
                finished = self.env.isFinished()
                if self.steps > self.delay_training:
                    avg_loss /= cnt
                    print(f"Episode: {episode}, Reward: {totreward:.1f}, invalid moves: {invalid_moves}, avg loss: {avg_loss:.3f}, eps: {self.temp:.3f}, dropout: {self.dropoutRate:.3f}, beta: {self.memory.beta:.3f}")
                else:
                    avg_loss = 0
                    print(f"Pre-Training... Episode {episode}")
                if episode == 0:
                    tf.summary.trace_on()
                    self.primary_network.traceable(self.agent.processState(1, last_state[1][0],self.to_train, 0).reshape(1,-1))
                with self.train_writer.as_default():
                    tf.summary.scalar('reward', totreward, step=episode)
                    tf.summary.scalar('avg loss', avg_loss, step=episode)
                    tf.summary.scalar('invalid moves', invalid_moves, step=episode)
                    if episode == 0:
                        tf.summary.trace_export('model', step=0)
                break
        return finished, reward
    def train(self, episodes: int):
        for index in range(episodes):
            self.gameLoop(index)
        self.primary_network.save_weights(f"models/" + self.store_path, save_format="tf")
    

#### State

In [57]:
class State(object):
    def __init__(self, state, valid_moves, terminal, env, last_move = None, parent = None):
        self.last_move = copy.deepcopy(last_move)
        self.n = 0
        self.q = 0
        self.terminal = copy.deepcopy(terminal)
        self.valid_moves = copy.deepcopy(valid_moves)
        self.untried_actions: list = list(self.valid_moves)
        self.state = copy.deepcopy(state)
        self.parent = parent
        self.children = []
        self.results = defaultdict(float)
        self.env: MillEnv = copy.deepcopy(env)
    def is_fully_expanded(self):
        return len(self.untried_actions) == 0
    def best_child(self, c_param=1.4):
        choices_weights = [
            (c.q / c.n) + c_param * np.sqrt((2 * np.log(self.n) / c.n))
            for c in self.children
        ]
        return self.children[np.argmax(choices_weights)]

    def rollout_policy(self):
        return self.valid_moves[np.random.randint(len(self.valid_moves))]
    def is_terminal_node(self):
        return self.terminal
    def expand(self):
        action = self.untried_actions.pop()
        self.env.setFullState(self.state[0], self.state[1],self.state[2], self.state[3],self.state[4], self.state[5],self.state[6], self.state[7], self.state[8])
        self.env.makeMove(action)
        next_state = self.env.getFullState()
        child_node = State(next_state, self.env.getValidMoves(), self.env.isFinished(), copy.deepcopy(self.env),action, parent=self)
        self.children.append(child_node)
        return child_node
    def rollout(self, discount, max_depth = None):
        self.env.setFullState(self.state[0], self.state[1],self.state[2], self.state[3],self.state[4], self.state[5],self.state[6], self.state[7], self.state[8])
        reward = 0
        winner = 0
        iteration = 0
        if max_depth is not None:
            while self.env.isFinished() == 0 and iteration < max_depth:
                last_player = self.env.isPlaying
                last_state = self.env.getSummary(last_player)
                last_move = self.env.moveNeeded
                action = self.env.getValidMoves()[np.random.randint(len(self.env.getValidMoves()))]
                self.env.makeMove(action)
                if last_player == self.state[1]:
                    reward += math.pow(discount, iteration) * self.env.getReward(last_state, self.env.getSummary(last_player), True,last_move)
                else:
                    reward -= 0.6 * (math.pow(discount, iteration) * self.env.getReward(last_state, self.env.getSummary(last_player), True,last_move))
                iteration += 1
        else:
            while self.env.isFinished() ==0:
                last_player = self.env.isPlaying
                last_state = self.env.getSummary(last_player)
                last_move = self.env.moveNeeded
                action = self.env.getValidMoves()[np.random.randint(len(self.env.getValidMoves()))]
                self.env.makeMove(action)
                if last_player == self.state[1]:
                    reward += math.pow(discount, iteration) * self.env.getReward(last_state, self.env.getSummary(last_player), True,last_move)
                else:
                    reward -= 0.6 * (math.pow(discount, iteration) * self.env.getReward(last_state, self.env.getSummary(last_player), True,last_move))
        if self.env.isFinished() == self.state[1]:
            reward += 5
            winner = 1* self.state[1]
        elif self.env.isFinished() == -self.state[1]:
            reward -= 5
            winner = -1* self.state[1]
        return reward, winner

    def backpropagate(self, result, winner):
        self.n += 1.
        self.q += result
        self.results[winner] += 1.
        if self.parent:
            self.parent.backpropagate(result, winner)

#### Monte Carlo Tree Search

In [55]:
class MonteCarloTreeSearch(object):
    def __init__(self, node):
        self.root: State = node
    def best_action(self, gamma, simulations_number = None, duration = None, max_depth = None) -> State:
        """
        get best action of state

        :parameter gamma: the discount factor
        :parameter max_depth: depth to search actions
        :parameter simulations_number : int number of simulations performed to get the best action
        :parameter duration: time to search

        :returns best child state
        """
        if simulations_number is not None:
            for _ in range(simulations_number):
                v = self._tree_policy(max_depth)
                reward, winner = v.rollout(gamma, max_depth)
                v.backpropagate(reward, winner)
        elif duration is not None:
            startTime = time.time()
            while time.time() - startTime <= duration:
                v = self._tree_policy(max_depth)
                reward, winner = v.rollout(gamma, max_depth)
                v.backpropagate(reward, winner)
        # to select best child go for exploitation only
        return self.root.best_child(c_param=0.)

    def setNewRoot(self, node):
        self.root = node
    def _tree_policy(self, max_depth = None):
        """
        selects node to run rollout/playout for
        Returns
        -------
        """

        current_node = self.root
        if max_depth is not None:
            itern  = 0
            while not current_node.is_terminal_node() and itern < max_depth:
                if not current_node.is_fully_expanded():
                    return current_node.expand()
                else:
                    current_node = current_node.best_child()
                itern += 1
        else:
            while not current_node.is_terminal_node():
                if not current_node.is_fully_expanded():
                    return current_node.expand()
                else:
                    current_node = current_node.best_child()
        return current_node



### Tests

#### Environment

In [44]:
envis = MillEnv()
print(envis.getInRows(14))

envis.makeMove(0)
envis.makeMove(3)
envis.makeMove(2)
envis.makeMove(4)
envis.makeMove(1)
envis.makeMove(3)
envis.makeMove(3)
print(envis.getValidMoves())
print(envis.getBoard())
print(envis.moveNeeded)
ThreeInChosenRow = abs(envis.board[envis.getInRows(0)].sum(axis=1)) == 3
print(~ThreeInChosenRow.any())
print(envis.getMoveFields(4))
print(envis.getSummary(1))

[[12 13 14]
 [ 2 14 23]]
[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[ 1.  1.  1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.]
0
False
[ 1.  0.  0. -1.]
(1, 0, 0, 1)


#### Agent

In [45]:
envi = MillEnv()
ag = Agent()
print(ag.processState(1,envi.getBoard(), 2,3).shape)
firstTime = time.time()
iters = 100
for i in range(iters):
    envi.reset()
    while envi.isFinished() == 0:
        validNow = envi.makeMove(ag.getPos(envi.getBoard(), 0, envi.getValidMoves()))
        if not validNow:
            print("invalid")
    print(f"Gewonnen hat {envi.isFinished()}")
print(time.time()- firstTime)
print((time.time()-firstTime)/iters)

(96,)
Gewonnen hat 1
Gewonnen hat 1
Gewonnen hat 1
Gewonnen hat 1
Gewonnen hat 1
Gewonnen hat 1
Gewonnen hat -1
Gewonnen hat 1
Gewonnen hat -1
Gewonnen hat -1
Gewonnen hat 1
Gewonnen hat 1
Gewonnen hat -1
Gewonnen hat 1
Gewonnen hat -1
Gewonnen hat 1
Gewonnen hat -1
Gewonnen hat -1
Gewonnen hat 1
Gewonnen hat -1
Gewonnen hat 1
Gewonnen hat 1
Gewonnen hat -1
Gewonnen hat -1
Gewonnen hat -1
Gewonnen hat 1
Gewonnen hat 1


KeyboardInterrupt: 

#### Displayer

In [ ]:
displayer = MillDisplayer()
displayer.windowsLoop()

#### Moderated  Play

In [ ]:
moderateGraphics = ModeratedGraphics()
moderateGraphics.playLoop()

#### Controller

In [ ]:
cont = Controler(0.05,0.4,70000,0.7,4,90000,0.4,1,60000, 0.9, 64,0.1,24,100000,f"Move0-{dt.datetime.now().strftime('%d%m%Y%H%M')}", 256,100,0)
cont.train(2000)


#### Monte Carlo

In [60]:
MCGraphics = ModeratedGraphics(monteCarlo=True, gamma=0.7)
MCGraphics.playLoop()